# 521153S Deep Learning Final Project

In [22]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights, vgg11, vgg13, vgg16, vgg19, VGG11_Weights, VGG13_Weights, VGG16_Weights, VGG19_Weights
import gdown
import urllib
import os
import random
import tarfile
import time
import shutil
import logging as lg
from tqdm.notebook import tqdm
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [23]:
# Set the logging level
# Possible Levels: DEBUG, INFO,
lg.basicConfig(level=lg.DEBUG)

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

Device: cuda:0


In [25]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

In [26]:
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'
cub_url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'
cub_file = './data/cub_200_2011.tgz'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

if not os.path.exists(cub_file):
    print('Downloading cub200_2011.tgz')
    response = urllib.request.urlretrieve(cub_url, cub_file)
    tar = tarfile.open(cub_file, 'r')
    for item in tar:
        tar.extract(item, './data')
    tar.close()


In [27]:
model_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
model_resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
model_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
model_resnet101 = resnet101(weights=ResNet101_Weights.DEFAULT)
model_resnet152 = resnet152(weights=ResNet152_Weights.DEFAULT)

model_vgg11 = vgg11(weights=VGG11_Weights.DEFAULT)
model_vgg13 = vgg13(weights=VGG13_Weights.DEFAULT)
model_vgg16 = vgg16(weights=VGG16_Weights.DEFAULT)
model_vgg19 = vgg19(weights=VGG19_Weights.DEFAULT)

In [ ]:
"""
train_dataset = torchvision.datasets.ImageFolder(
    root='./data/train',
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
    ])
)


test_dataset = torchvision.datasets.ImageFolder(
    root='./data/test',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/val',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)


dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
dataloader_validation = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
"""

In [28]:
# Stratified split, tries to maintain a proportional distribution of samples for each class in all three sets (train, val and test).

# Val and Test ratio total, Train ratio (1-test_ratio)
test_ratio = 0.3

# Split between Val and Test
test_val_split = 0.5

# Loading the ImageFolder dataset
train_dataset = torchvision.datasets.ImageFolder(root='./data/train')

# Getting the number of classes
num_classes = len(train_dataset.classes)

# Initializing lists to store indices for train, validation, and test sets
train_indices, val_indices, test_indices = [], [], []

# Splitting each class separately
for class_index in range(num_classes):
    # Getting indices for samples belonging to the current class
    class_indices = np.where(np.array(train_dataset.targets) == class_index)[0]
    
    # Splitting indices for each class separately into train, validation, and test sets
    train_idx, temp_idx = train_test_split(class_indices, test_size=test_ratio, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=test_val_split, random_state=42)
    
    # Extending the lists to store indices for each set
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)
    test_indices.extend(test_idx)

# Creating Subset objects using the selected indices for each set
train_set = Subset(train_dataset, train_indices)
val_set = Subset(train_dataset, val_indices)
test_set = Subset(train_dataset, test_indices)

# Transform functions
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),      
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

train_set.dataset.transform = transform_train
val_set.dataset.transform = transform_eval
test_set.dataset.transform = transform_eval

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Printing the sizes of the resulting sets
print(f"Number of samples in the training set: {len(train_set)}")
print(f"Number of samples in the validation set: {len(val_set)}")
print(f"Number of samples in the test set: {len(test_set)}")

Number of samples in the training set: 26880
Number of samples in the validation set: 5760
Number of samples in the test set: 5760


In [29]:
# Define the used model
model = model_resnet18
model.name = "resnet18"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [30]:
def evaluate(model, data_loader, device, criterion):

    model.eval() 
    # valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    num_images = 0.0 # used to accumulate number of images
    
    #with torch.no_grad():
    for i, data in enumerate(data_loader, 0):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = criterion(outputs, labels)
        # valid_running_loss += loss.item()
        # Calculate the accuracy.
        #preds = outputs.argmax(dim=1)
        preds = torch.max(outputs, 1).indices
        
        #valid_running_correct += preds.eq(labels).sum()
        valid_running_correct += torch.sum(preds == labels.data)
        num_images += len(labels)
        
    # Loss and accuracy for the complete epoch.
    # epoch_loss = valid_running_loss / counter
    lg.debug(f"EVAL:   loss.item(): {loss.item()} valid_running_correct: {valid_running_correct.item()}")

    epoch_acc = valid_running_correct / num_images
    return epoch_acc

In [31]:
def train(model, trainLoader, valLoader, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_loss = []
    train_acc = []

    # Start the training.
    if torch.cuda.is_available():
        print("Using CUDA")

    print('Training')
    
    lg.debug(f" len(trainLoader.dataset): {len(trainLoader.dataset)}")
    lg.debug(f" len(valLoader.dataset): {len(valLoader.dataset)}")

    
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        num_images = 0.0 # used to accumulate number of images
        for i, data in enumerate(trainLoader,0):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += torch.sum(preds == labels.data)
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()    # Output training 
            num_images += len(image)
            if counter % 100 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Step: {counter}/{len(trainLoader)}, Loss: {loss.item():.4f}')
        
        # reduce learning rate every 10 epochs by factor of 10
        if (epoch+1) % 10 == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 10.0


        # Validation
        valid_epoch_acc = evaluate(model, valLoader, device, criterion)
        lg.debug(f"TRAIN: loss.item(): {loss.item()}, train_running_correct: {train_running_correct}")
        
        # train_epoch_loss = train_running_loss / counter
        # train_loss.append(train_epoch_loss)
        # valid_loss.append(valid_epoch_loss)
        train_epoch_acc = train_running_correct  / num_images
        # train_acc.append(train_epoch_acc )

        print('Epoch: %d/%d, Training accuracy: %f, Loss: %f, Validation accuracy: %f' % (epoch+1, num_epochs, train_epoch_acc, loss.item(), valid_epoch_acc))

        # valid_acc.append(valid_epoch_acc)
        # lg.debug(f"Training loss mean: {train_epoch_loss :.3f}")
        # print(f"Validation acc: {valid_epoch_acc:.3f}")
        print('-'*50)


    return model

In [ ]:
# pretrained_model = train(model, dataloader_train, dataloader_validation, criterion, optimizer, num_epochs, device)

In [32]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

DEBUG:root: len(trainLoader.dataset): 26880
DEBUG:root: len(valLoader.dataset): 5760


Using CUDA
Training
Epoch: 1/10, Step: 100/210, Loss: 2.8310
Epoch: 1/10, Step: 200/210, Loss: 1.7128


DEBUG:root:EVAL:   loss.item(): 2.3583860397338867 valid_running_correct: 4034
DEBUG:root:TRAIN: loss.item(): 1.6559829711914062, train_running_correct: 11243


Epoch: 1/10, Training accuracy: 0.418266, Loss: 1.655983, Validation accuracy: 0.700347
--------------------------------------------------
Epoch: 2/10, Step: 100/210, Loss: 1.1455
Epoch: 2/10, Step: 200/210, Loss: 0.9268


DEBUG:root:EVAL:   loss.item(): 1.2889606952667236 valid_running_correct: 4510
DEBUG:root:TRAIN: loss.item(): 1.123974084854126, train_running_correct: 20419


...

Epoch: 8/10, Training accuracy: 0.934784, Loss: 0.257915, Validation accuracy: 0.847049
--------------------------------------------------
Epoch: 9/10, Step: 100/210, Loss: 0.2704
Epoch: 9/10, Step: 200/210, Loss: 0.3734


DEBUG:root:EVAL:   loss.item(): 0.6640430688858032 valid_running_correct: 4898
DEBUG:root:TRAIN: loss.item(): 0.16336844861507416, train_running_correct: 25488


Epoch: 9/10, Training accuracy: 0.948214, Loss: 0.163368, Validation accuracy: 0.850347
--------------------------------------------------
Epoch: 10/10, Step: 100/210, Loss: 0.1529
Epoch: 10/10, Step: 200/210, Loss: 0.2073


DEBUG:root:EVAL:   loss.item(): 0.6935462951660156 valid_running_correct: 4902
DEBUG:root:TRAIN: loss.item(): 0.19226767122745514, train_running_correct: 25786


Epoch: 10/10, Training accuracy: 0.959301, Loss: 0.192268, Validation accuracy: 0.851042
--------------------------------------------------


In [33]:
def save_model(trained_model):
    # Create Models Folder
    if not os.path.exists('./models'):
        print('Creating models directory')
        os.mkdir('./models')

    # Save the model
    if not os.path.exists(f'./models/{trained_model.name}.pth'):
        print(f'Saving Model {trained_model.name}')
        torch.save(trained_model, f'./models/{trained_model.name}.pth')

def load_model(model_name):
    if not os.path.exists(f'./models/{model_name}.pth'):
        print(f'Cannot Load {model_name}')
        return None
    print(f'Loading Model {model_name}')
    model = torch.load(f'./models/{model_name}.pth')
    model.eval()
    return model

In [34]:
save_model(pretrained_model)

In [35]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

DEBUG:root:EVAL:   loss.item(): 0.8225520849227905 valid_running_correct: 4919


test acc: 0.854


In [37]:
def choose_100_eurosat():
    '''
    The `choose_100_eurosat()` function selects 100 images from the EuroSAT dataset and copies them to the `data/eurosat_validation` folder. It also creates a `data/eurosat_training` folder if it doesn't already exist.
    '''
    if not os.path.exists('./data/eurosat_validation'):
        print('Creating data directory')
        os.mkdir('./data/eurosat_validation')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_validation'))
        os.mkdir('./data/eurosat_validation')

    if not os.path.exists('./data/eurosat_training'):
        print('Creating training directory')
        os.mkdir('./data/eurosat_training')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_training'))
        os.mkdir('./data/eurosat_training')

    # Load the EuroSAT Categories
    eurosat_categories = [name for name in os.listdir('./data/eurosat/EuroSAT_RGB/') if os.path.isdir(os.path.join('./data/eurosat/EuroSAT_RGB/', name))]
    # Randomly select 5 categories
    lg.debug(f"Selecting 5 categories from {eurosat_categories} categories")
    selected_categories = np.random.choice(eurosat_categories, 5, replace=False)
    lg.debug(f"Selected categories: {selected_categories}")

    selected_images = []
    training_images = []

    # From each directory, randomly select 20 images
    for category in selected_categories:
        images = os.listdir(os.path.join('./data/eurosat/EuroSAT_RGB/', category))
        selected = random.sample(images, 20)
        selected_images.extend([(category, image) for image in selected])
        lg.debug(f"Selected {selected} from {category}")
        # Copy selected images to the selected directory
        for image in selected:
            if not os.path.exists(f"./data/eurosat_validation/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_validation/{category}")
            shutil.copyfile(os.path.join('./data/eurosat/EuroSAT_RGB', category, image), os.path.join(f"./data/eurosat_validation/{category}", image))

    # From these 100 images, randomly select 5 images from each category for the training set
    for category in selected_categories:
        category_images = [image for (cat, image) in selected_images if cat == category]
        training = random.sample(category_images, 5)
        training_images.extend(training)
        lg.debug(f"Selected {training} for training")

        # Copy training images to the training directory
        for image in training:
            if not os.path.exists(f"./data/eurosat_training/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_training/{category}")
            shutil.move(os.path.join(f"./data/eurosat_validation/{category}", image), os.path.join(f"./data/eurosat_training/{category}", image))

In [38]:
batch_size = 12

In [39]:
def load_euro_datasets():

    # Choose 100 images from EuroSAT dataset
    choose_100_eurosat()

    eurosat_train_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_training',
        transform=transforms.Compose([
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
        ])
    )

    eurosat_validation_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_validation',
        transform=transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    )

    eurosat_dataloader_train = DataLoader(eurosat_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    eurosat_dataloader_validation = DataLoader(eurosat_validation_dataset, batch_size=batch_size, shuffle = False, num_workers=num_workers)

    return eurosat_dataloader_train, eurosat_dataloader_validation

In [40]:
# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False

model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet18_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

# Train the model on the EuroSAT dataset
eurosat_trained_model = train(model_resnet18_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)


# Evaluate the model on the EuroSAT dataset
i = 10
for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    eurosat_trained_model = train(eurosat_trained_model, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(eurosat_trained_model, eurosat_dataloader_validation, device, criterion)
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{eurosat_trained_model.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        f.close()

DEBUG:root:Selecting 5 categories from ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'] categories
DEBUG:root:Selected categories: ['Residential' 'PermanentCrop' 'HerbaceousVegetation' 'River' 'SeaLake']
DEBUG:root:Selected ['Residential_1541.jpg', 'Residential_40.jpg', 'Residential_158.jpg', 'Residential_1945.jpg', 'Residential_1617.jpg', 'Residential_1105.jpg', 'Residential_66.jpg', 'Residential_193.jpg', 'Residential_424.jpg', 'Residential_1889.jpg', 'Residential_2951.jpg', 'Residential_2.jpg', 'Residential_2690.jpg', 'Residential_423.jpg', 'Residential_1132.jpg', 'Residential_2785.jpg', 'Residential_1381.jpg', 'Residential_1046.jpg', 'Residential_1532.jpg', 'Residential_2700.jpg'] from Residential
DEBUG:root:Creating Residential directory
DEBUG:root:Selected ['PermanentCrop_287.jpg', 'PermanentCrop_2308.jpg', 'PermanentCrop_715.jpg', 'PermanentCrop_290.jpg', 'PermanentCrop_702.jpg', 'PermanentCr

Loading Model resnet18
Using CUDA
Training


DEBUG:root:EVAL:   loss.item(): 1.6423953771591187 valid_running_correct: 14
DEBUG:root:TRAIN: loss.item(): 2.4552183151245117, train_running_correct: 5


Epoch: 1/30, Training accuracy: 0.200000, Loss: 2.455218, Validation accuracy: 0.186667
--------------------------------------------------


DEBUG:root:EVAL:   loss.item(): 1.841840386390686 valid_running_correct: 15
DEBUG:root:TRAIN: loss.item(): 1.1028521060943604, train_running_correct: 6


...

Epoch: 28/30, Training accuracy: 0.920000, Loss: 3.524022, Validation accuracy: 0.760000
--------------------------------------------------


DEBUG:root:EVAL:   loss.item(): 0.1097554937005043 valid_running_correct: 52
DEBUG:root:TRAIN: loss.item(): 2.8431854248046875, train_running_correct: 22


Epoch: 29/30, Training accuracy: 0.880000, Loss: 2.843185, Validation accuracy: 0.693333
--------------------------------------------------


DEBUG:root:EVAL:   loss.item(): 0.041999027132987976 valid_running_correct: 43
DEBUG:root:TRAIN: loss.item(): 3.1651673316955566, train_running_correct: 19


Epoch: 30/30, Training accuracy: 0.760000, Loss: 3.165167, Validation accuracy: 0.573333
--------------------------------------------------


## RESNET 34

In [ ]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

In [ ]:
# Define the used model
model = model_resnet34
model.name = "resnet34"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
save_model(pretrained_model)

In [ ]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

In [ ]:
batch_size = 12

In [ ]:
# Load the pretrained model
model_resnet34_fine = load_model("resnet34")

for param in model_resnet34_fine.parameters():
    param.requires_grad = False

model_resnet34_fine.fc = nn.Linear(model_resnet34_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet34_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

eurosat_trained_model = train(model_resnet34_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)


In [ ]:
i = 11

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    eurosat_trained_model = train(eurosat_trained_model, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(eurosat_trained_model, eurosat_dataloader_validation, device, criterion)
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{eurosat_trained_model.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        f.close()

## RESNET 50

In [ ]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2


# Define the used model
model = model_resnet50
model.name = "resnet50"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
save_model(pretrained_model)

# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

In [ ]:
batch_size = 12


# Load the pretrained model
model_resnet50_fine = load_model("resnet50")

for param in model_resnet50_fine.parameters():
    param.requires_grad = False

model_resnet50_fine.fc = nn.Linear(model_resnet50_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet50_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

eurosat_trained_model = train(model_resnet50_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)

In [ ]:
i = 11
j = 1

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    eurosat_trained_model = train(eurosat_trained_model, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(eurosat_trained_model, eurosat_dataloader_validation, device, criterion)
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        f.close()

## DATASET: CUB

In [42]:
batch_size = 16

In [51]:
cub_dataset = torchvision.datasets.ImageFolder(root='./data/CUB_200_2011/images')

# Transform functions
transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

cub_dataset.transform = transform_eval

# Loading the ImageFolder dataset


# Getting the number of classes
cub_num_classes = len(cub_dataset.classes)

cub_val_loader = DataLoader(cub_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Printing the sizes of the resulting sets
print(f"Number of samples in the validation set: {len(cub_dataset)}")


Number of samples in the validation set: 11788


In [52]:
# Batch size during training
num_workers = 1

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2


# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False


model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet18_fine.parameters(), lr=0.001, momentum=0.9)

model_resnet18_fine.to(device)

if torch.cuda.is_available():
    print("Using CUDA")

cub_model = evaluate(model_resnet18_fine, cub_val_loader, device, criterion)

Loading Model resnet18


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
